In [ ]:
from functions import get_data
import pandas as pd
import numpy as np


In [ ]:
us_data=get_data('GOLD')

In [ ]:
def x_points(data,index):
    data['mtime'] = pd.to_datetime(data['time'], unit='s')
    start_date = data['mtime'][index].date()
    t=data['mtime'][index].time()
    print(f'start time {t}')
    print(f'start date{start_date}')
    filtered_data = data[(data['mtime'].dt.date == start_date) & (data.index <= index)]
    # print(filtered_data)
    # print(f'filtered_data:\n{filtered_data.head()}')
    max_value = filtered_data['Close'].max()
    min_value = filtered_data['Close'].min()
    return max_value, min_value
    
    

In [ ]:
us_data['mtime']=pd.to_datetime(us_data['time'],unit='s')
us_data['mtime']

In [ ]:
h,j=x_points(us_data,40000)
print(h,j)

In [ ]:
def distance_checker(price,value,lim):
    plh=price-value
    plh=abs(plh)
    if plh >lim:
        return 1
    else:
        return 0
    
    

In [ ]:
def find_status(data,index,window,size):
    cspreads=[]
    for i in range(window):
        datapoint=data[index-i]
        close=datapoint['Close']
        openv=datapoint['Open']
        cspread=abs(close-openv)
        cspreads.append(cspread)
    if max(cspreads)>size:
        return 1
    else:
        return 0
    

In [ ]:
from functions import calc_prof

In [ ]:
def daily(data,starttime,adjtime,lim,window,c_size,sl_adj,multiplier):
    trades=[]
    for i in range(len(data)):
        time=data['mtime'][i].time
        adj_time = pd.to_datetime(adjtime).time()# adtime being such astring '03:00:00'
        if time<adj_time:
            continue
        else:
            trade=0
            tday=data['date'][i]
            target_datetime = pd.to_datetime(f'{tday} {starttime}')
            val=data[data['mtime'] == target_datetime]['value'].values
            startindex=val.index
            begin_max_value,begin_min_value=x_points(data,startindex)
            maxv=begin_max_value
            minv=begin_min_value
            opendate=None
            openprice=0
            tp=0
            sl=0
            price=data['Close'][i]
            if trade==0:
                if price>maxv:
                    if distance_checker(price,maxv,lim)==0:
                        maxv=price
                        continue
                    elif distance_checker(price,maxv,lim)==1:
                        if find_status(data,i,window,c_size)==1:
                            trade=-1
                            openprice=price
                            opendate=data['time'][i]
                            lowp=data['Low'][i]
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            continue
                        else:
                            continue
                            
                elif price<minv:
                    if distance_checker(price,minv,lim)==0:
                        minv=price
                        continue
                    elif distance_checker(price,minv,lim)==1:
                        if find_status(data,i,window,c_size)==1:
                            trade=1
                            openprice=price
                            opendate=data['time'][i]
                            highp=data['High'][i]
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
                            
                            continue
                        else:
                            continue
            if trade==1:
                if price>tp:
                    
                    closedate=data['time'][i]     
                    profit=calc_prof(price,openprice)
                    trades.append('Buy',opendate,closedate,profit)
                    
                    trade=0
                    continue
                elif price<sl:
                    closedate=data['time'][i]
                    profit=calc_prof(price,openprice)
                    trades.append('Buy',opendate,closedate,profit)
                    
                    trade=0
                    continue
            elif trade==-1:
                if price<tp:
                    closedate=data['time'][i]
                    profit=calc_prof(openprice,price)
                    trades.append('Sell',opendate,closedate,profit)
                    
                    trade=0
                    continue
                elif price>sl:
                    closedate=data['time'][i]
                    profit=calc_prof(openprice,price)
                    trades.append('Sell',opendate,closedate,profit)
                    
                    trade=0
                    continue
    return trades
                    
                
        
        
        
        

In [ ]:
testing_dt=us_data[400:10000]

In [ ]:
y=daily(testing_dt,'03:00:00','03:00:00',3.0,4,4.5,2.5,2.5)